<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Construct a Custom Coadded Image</b> <br>
Contact author: Melissa Graham <br>
Last verified to run: 2024-10-21 <br>
LSST Science Pipelines version: <b>Weekly 2022_40</b> <br>
Container Size: large <br>
Targeted learning level: advanced <br>

___

**WARNING:
This notebook will only run with LSST Science Pipelines version Weekly 2022_40.**

To find out which version of the LSST Science Pipelines you are using, look in the footer bar or execute the cell below.

In [ ]:
! echo $IMAGE_DESCRIPTION

If you are using `w_2022_40`, you may proceed with executing the tutorial.

If you are **not** using `w_2022_40` you **must** log out and start a new server:
 1. At top left in the menu bar choose File then Save All and Exit.
 2. Re-enter the Notebook Aspect.
 3. At <a href="https://dp0-2.lsst.io/data-access-analysis-tools/nb-intro.html#how-to-log-in-navigate-and-log-out-of-jupyterlab">the "Server Options" stage</a>, under "Select uncached image (slower start)" choose `w_2022_40`.
 4. Note that it might take a few minutes to start your server with an old image.

**Why do I need to use an old image for this tutorial notebook?**
In this tutorial and in the future with real LSST data, users will be able to recreate coadds starting with intermediate data products (the warps).
On Feb 16 2023, as documented in the <a href="https://dp0-2.lsst.io/tutorials-examples/major-updates-log.html#major-updates-log">Major Updates Log</a> for DP0.2 tutorials, the recommended image of the RSP at data.lsst.cloud was bumped from Weekly 2022_40 to Weekly 2023_07.
However, the latest versions of the pipelines are not compatible with the intermediate data products of DP0.2, which were produced in early 2022.
To update this tutorial to be able to use Weekly 2023_07, it would have to demonstrate how to recreate coadds *starting with the raw data products*.
This is pedagogically undesirable because it does not accurately represent *future workflows*, which is the goal of DP0.2.
Thus, it is recommended that delegates learn how to recreate coadds with Weekly 2022_40.

___


**NOTICE:
This notebook will only run with the butler repo alias `dp02-direct`.**

All other tutorial notebooks in this repository use the `dp02` alias, which was updated in Oct 2024 to provide
read-only access to butler data repos.
As this tutorial writes processed images back to the butler, "direct access" is needed and `dp02-direct` must be used.
Note that this is a temporary work around for Data Preview 0. 
Butler functionality for Data Preview 1 will be different, as the butler service continues to evolve.

___

<br><br><br><br>

**Description:** Create a custom "deepCoadd" using only a subset of the input visits.

**Skills:** Use of pipetasks for image coaddition. Creating and writing to Butler collections. Properties of deepCoadds.

**LSST Data Products:** visitTable, deepCoadd images

**Packages:** lsst.daf.butler, lsst.ctrl.mpexec, lsst.pipe.base

**Credit:** Originally developed by Melissa Graham and Clare Saunders.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

# 1. Introduction

This notebook shows how to retrieve information about the individual images that contributed to a deepCoadd, and how to make a new "custom coadd" image using only a subset of the inputs.

In the past you might have used IRAF's imcombine or AstrOmatic's SWarp (for example) to coadd images.
This notebook demonstrates the appropriate method for coadding LSST images with the LSST Science Pipelines.

Science applications of coadding a subset of LSST images include searching for faint, slowly-evolving transients or variables (e.g., coadding images by season), using the effects of differential chromatic refraction (e.g., coadding images in bins of airmass), or searching for low surface brightness features (e.g., coadding only dark-time images with the faintest sky backgrounds).

**Adopted science use case for custom coadds: supernova precursor events.**<br>
This notebook tutorial uses the science case of a hypothetical supernova that occurred at RA 55.745834, Dec -32.269167 on MJD = 60960.
(There is no DC2 supernova there, we just use the coordinates as an example).
Since the deepCoadds are made out of years worth of images, and are combined to remove most transient flux, the supernova itself would not appear in the deepCoadd.
However, some supernovae are known to have faint precursor eruptions, and characterizing precursor events is useful for understanding the supernova progenitor star and, e.g., its pre-supernova mass-loss episodes.
It could be possible to detect faint precursor eruptions by stacking images obtained prior to the supernova explosion.

**Caveats for the adopted science use case.**<br>
1. This science use case of supernova precurors would be much better achieved by stacking the difference images. However, we wanted to make this a tutorial that creates custom coadds, and our science background is supernovae. So there remains a bit of a mismatch between science use case and image type for this tutorial.

2. This tutorial uses only i-band visits to recreate coadds, but in reality scientists looking for supernova precursor events would use all filters (searching the bluer filters in particular would be more relevant).

3. This tutorial uses images obtained in a one-month window prior to the supernova, but in reality scientists looking for supernova precursor events would use more windows, further in the past. That is left as an exercise for the learner in Section 5.

4. The simulated DC2 supernovae were all of Type Ia and no precursor events were simulated.

**Source detection in custom coadds.**<br>
See the next tutorial notebook in the series, 09b_Custom_Coadd_Sources.ipynb, to learn how to run source detection and measurement on the new custom coadd.

## 1.1. Package imports

Standard python and astropy packages for numerical processing, plotting, databases, unit conversion, etc.

LSST Science Pipelines packages for data access, display, sky coordinates, and pipeline tasks.

In [ ]:
import getpass
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas
from astropy.time import Time

import lsst.geom
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler, DatasetType, CollectionType
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

## 1.2. Define functions and parameters

Set a few parameters related to plotting and display.

In [ ]:
font = {'size': 14}
matplotlib.rc('font', **font)

pandas.set_option('display.max_rows', 1000)

afwDisplay.setDefaultBackend('matplotlib')

Set the DP0.2 config and collection, and instantiate a butler.

In [ ]:
butler = Butler('dp02-direct', collections='2.2i/runs/DP0.2')

**Option:** display all the `deepCoadd` `datasetTypes` available via the butler.

In [ ]:
# for x in sorted(butler.registry.queryDatasetTypes('deepCoadd*')):
#     print(x)

# 2. Identify the visits to combine

This example starts with a given sky coordinate. In the adopted science use case for this tutorial, consider this the coordinates of a supernova. (There is no DC2 supernova at this location, but the coordinates were chosen to be near a rich galaxy cluster, as a visually interesting target for a custom coadd).

The DC2 skyMap is used to identify the i-band `deepCoadd` which contains that coordinate, and then the butler is used to retrieve the `deepCoadd` and the list of visit ids which were combined to create it.

The `visitTable` is then used to obtain the acquisition dates of the input visits.
For this example, the visits in a short time range will be selected as the visits to be coadded.

## 2.1. Identify and retrieve the deepCoadd

Define the coordinates and filter to identify and retrieve the relevant `deepCoadd`.

This takes 4-5 seconds.

In [ ]:
my_ra_deg = 55.745834
my_dec_deg = -32.269167
my_filter = 'i'

my_spherePoint = lsst.geom.SpherePoint(my_ra_deg*lsst.geom.degrees, my_dec_deg*lsst.geom.degrees)

skymap = butler.get('skyMap')
tract = skymap.findTract(my_spherePoint)
my_tract = tract.tract_id
my_patch = tract.findPatch(my_spherePoint).getSequentialIndex()
print('My tract and patch: ', my_tract, my_patch)

my_dataId = {'band': my_filter, 'tract': my_tract, 'patch': my_patch}
my_deepCoadd = butler.get('deepCoadd', dataId=my_dataId)

del my_ra_deg, my_dec_deg, my_filter, my_spherePoint, skymap, tract

**Option:** display the `deepCoadd` image.

In [ ]:
# fig = plt.figure(figsize=(6, 4))
# afw_display = afwDisplay.Display(1)
# afw_display.scale('asinh', 'zscale')
# afw_display.mtv(my_deepCoadd.image)
# plt.gca().axis('off')

**Option:** learn more about the `deepCoadd` metadata, such as bounding box, corners, and the World Coordinate System (WCS), by uncommenting and executing the cell below.
It is not necessary to know the bounding box for a `deepCoadd` in order to find all of the calexps that were used to assemble it, this is simply a demonstration for the learner.

In [ ]:
# my_deepCoadd_bbox = butler.get('deepCoadd.bbox', dataId=my_dataId)
# print('bbox')
# print(my_deepCoadd_bbox.beginX, my_deepCoadd_bbox.beginY,
#       my_deepCoadd_bbox.endX, my_deepCoadd_bbox.endY)

# print('')
# print('corners')
# print(my_deepCoadd_bbox.getCorners())

# print('')
# print('wcs')
# my_deepCoadd_wcs = butler.get('deepCoadd.wcs', dataId=my_dataId)
# print(my_deepCoadd_wcs)

# del my_deepCoadd_bbox, my_deepCoadd_wcs

## 2.2. Retrieve the deepCoadd's input visits

In [ ]:
my_coadd_inputs = my_deepCoadd.getInfo().getCoaddInputs()

The length of this table, 161, indicates that 161 separate visits contributed to this `deepCoadd`.

In [ ]:
len(my_coadd_inputs.visits)

**Option:** display the information for the `deepCoadd` inputs as an astropy table.

In [ ]:
# my_coadd_inputs.visits.asAstropy()

**Option:** list the `id` of each of the `deepCoadd` input visits.

In [ ]:
# my_coadd_visits = my_coadd_inputs.visits['id']
# my_coadd_visits

> **Notice:** The `visitId` is a unique identifier for a visit, which is a full-focal plane exposure obtained at a specific time, at specific sky coordinates, in one filter. The `visitID` does not also uniquely identify the CCD (detector): that is called a `ccdVisitId`. 

## 2.3. Identify the acquisition dates for the input visits

First, get the entire visit table.

In [ ]:
visitTableRef = list(butler.registry.queryDatasets('visitTable'))

In [ ]:
visitTable = butler.get(visitTableRef[0])

**Option:** display the contents of the entire `visitTable`.

In [ ]:
# visitTable

Retrieve the Modified Julian Dates (MJDs; `expMidptMJD`) of our `deepCoadd`'s input visits.

The fact that the `id` column for both the `my_coadd_inputs.visits` table and the `visitTable` is the visit number makes doing this a simple single line of code.

In [ ]:
my_coadd_visits_mjds = visitTable.loc[my_coadd_inputs.visits['id']]['expMidptMJD']

This list of MJDs has 161 elements, for the 161 separate visits contributed to this `deepCoadd`.

In [ ]:
len(my_coadd_visits_mjds)

## 2.4. Identify input visits to combine into a new Coadd

For the sake of the adopted science use case, the supernova exploded on MJD = 60960.

As a user, we want to stack images first from the month before to look for a faint precursor, and then maybe also the month before that.

Thus, we identify input visits in two date ranges:

Window1: 60925 to 60955 (the month before)

Window2: 60885 to 60922 (the month before that)

In [ ]:
Window1_start = 60925
Window1_end = 60955

Window2_start = 60885
Window2_end = 60922

fig, ax = plt.subplots(2, figsize=(10, 8))

ax[0].axvline(60960, lw=1, ls='solid', color='black')
ax[0].hist(my_coadd_visits_mjds, bins=150, color='dodgerblue')
ax[0].set_xlabel('MJD')
ax[0].set_ylabel('Number of Visits')
ax[0].axvline(Window1_start, ls='dotted', color='firebrick')
ax[0].axvline(Window1_end, ls='dotted', color='firebrick')
ax[0].axvline(Window2_start, ls='dashed', color='darkorange')
ax[0].axvline(Window2_end, ls='dashed', color='darkorange')

ax[1].axvline(60960, lw=1, ls='solid', color='black')
ax[1].text(60960.5, 7.5, 'SN', color='black')
ax[1].hist(my_coadd_visits_mjds, bins=150, color='dodgerblue')
ax[1].set_xlabel('MJD')
ax[1].set_ylabel('Number of Visits')
ax[1].set_xlim([60880, 60985])
ax[1].axvline(Window1_start, ls='dotted', color='firebrick')
ax[1].axvline(Window1_end, ls='dotted', color='firebrick')
ax[1].text(Window1_start+1, 7.5, 'Window1', color='firebrick')
ax[1].axvline(Window2_start, ls='dashed', color='darkorange')
ax[1].axvline(Window2_end, ls='dashed', color='darkorange')
ax[1].text(Window2_start+1, 7.5, 'Window2', color='darkorange')

plt.show()

There are six visits in Window1, and ten in the earlier Window2.

# 3. Create a coadd for the visits in Window1

Put the list of visits for Window1 into a string, formatted as a tuple, for use in a query later on.
("Formatted as a tuple" means within round brackets and separated by commas).

> **Notice:** Changes to the `SimplePipelineExecutor` are planned so as to remove the need for users to reformat `visitId` lists as strings.

In [ ]:
my_range = np.array((my_coadd_visits_mjds > Window1_start)
                    & (my_coadd_visits_mjds < Window1_end))
my_visits = my_coadd_inputs.visits[my_range]
my_visits_tupleString = "("+",".join(my_visits['id'].astype(str))+")"
print(my_visits_tupleString)

## 3.1. Name a new butler collection for your output

Use the convention `u/<Your User Name>/<Collection Identifier>` to set up a new butler output collection for this tutorial.

For `<Collection Identifier>`, for the first test of this notebook creating a custom coadd for Window1, "custom_coadd_window1_test1".

The new `deepCoadd` generated by this tutorial will be stored in the butler, just like the DP0 data sets generated by Rubin staff.
Everything stored in the butler is visible to all users.
All users should feel free to go ahead and build up a bunch of processed data in the butler. Rubin staff will reach out if this ever gets to be "too much".

> **Warning:** For DP0, the butler is a shared data repository, and everything stored in it is accessible to everyone. For DP0, there are no safeguards preventing users from deleting other people's data. Thus, for DP0, we are not teaching users how to delete _anything_ from the butler.

In [ ]:
my_username = getpass.getuser()
print(my_username)

In [ ]:
my_collection_identifier = 'custom_coadd_window1_test1'
print(my_collection_identifier)

In [ ]:
my_outputCollection = 'u/'+my_username+'/'+my_collection_identifier
print('Name of new butler collection for my output: ', my_outputCollection)

Check if this output collection already exists.

If nothing is printed below this cell, the output collection does not already exist.

In [ ]:
for c in sorted(butler.registry.queryCollections()):
    if c.find(my_outputCollection) > -1:
        print(c)

Delete the current butler. In the next section, a "simple butler" is created and used thereafter.

In [ ]:
del butler

## 3.2. Set up a simple butler with your output collection

Set up a simple butler to use with the simple pipeline executor that is made in Section 3.6 below.

This simple butler defines the input collection that contains the images which will be used to create the custom coadd, `2.2i/runs/DP0.2`, and also the output collection that will contain the custom `deepCoadd`, `my_outputCollection`, as defined in Section 3.1 above.

In [ ]:
simpleButler = SimplePipelineExecutor.prep_butler('dp02-direct', 
                                                  inputs=['2.2i/runs/DP0.2'], 
                                                  output=my_outputCollection)

Below, check that the newly created output collection is first in the list.

> **Notice:** A run timestamp has been added to `my_outputCollection` as additional information for users.

> **Warning:** If you want to make multiple custom coadds, you should identify each with a new output collection name, such as `custom_coadd_window1_test2` or `custom_coadd_window2_test1`, and so on. <br>
Note that re-executing Section 3 with the _same output collection name_ will produce results with a new run timestamp, but the butler always retrieves data _from the most recent timestamp for a given collection_. Not setting a new output collection name for a new custom coadd is essentially like "overwriting" your results in the butler. It is not recommended to work that way, but to bookkeep using output collection names.

In [ ]:
simpleButler.registry.getCollectionChain(my_outputCollection)

**Option:** check the output collection run names that currently exist.

In [ ]:
# for c in sorted(simpleButler.registry.queryCollections(my_outputCollection+'*')):
#     print('Found: ', c)

## 3.3. Create a pipeline with the makeWarp and assembleCoadd tasks

In order to combine the identified visits into a new `deepCoadd`, two steps of the larger Data Release Pipeline (DRP) must be included: `makeWarp` and `assembleCoadd`. 
Although warped images are created during processing, they are not stored long-term because they take up a lot of space and because they can be easily recreated when needed.

For more information, review the relevant documentation about <a href="https://pipelines.lsst.io/v/weekly/modules/lsst.pipe.base/creating-a-pipeline.html">creating a pipeline</a> or the <a href="https://pipelines.lsst.io/modules/lsst.pipe.tasks/tasks/lsst.pipe.tasks.assembleCoadd.AssembleCoaddTask.html">assembleCoadd<a> task.

**The `from_uri` method.**
    
The following method for creating the `assembleCoaddPipeline` task in a notebook (i.e., using the `from_uri` function and passing a file path) is not intuitive, but is preferred because it gives the user all of the configuration settings for the instrument automatically, and because it most closely replicates the command-line syntax.
    
A URI is a uniform resource identifier, and as seen below it can take both a file path and additional information: in this case, the additional information is the processing steps that we wish to use, which are defined in the file.

**The "yaml" file.**
    
yaml is a human-readable data-serialization language.
It is commonly used for configuration files and in applications where data are being stored or transmitted. 
    
All the DP0 tasks are listed in the DPO data release production pipeline definition yaml.
To see this full list of tasks, first open a new terminal (click the blue + button at upper left and then select terminal).
Then create a Rubin Observatory environment, and render the pipeline yaml content via the `pipetask build` command:
> `setup lsst_distrib` <br>
> `pipetask build -p $DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml --show pipeline`<br>

You will see quite a lot of yaml output as a result of this command. For more information about `pipetask build`, please check out the  <a href="https://dp0-2.lsst.io/tutorials-examples/cmdline-custom-coadd.html">command line version</a> of this custom coadd tutorial. Note that, witihin the `pipetask build` yaml output, the ordering of tasks within a given step may be randomized.

You can see an abbreviated version of the DP0 pipeline definition yaml by isolating "step3" as follows, where "step3" refers to coaddition:
    
> `pipetask build -p $DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml#step3 --show pipeline`<br>
    
The above command assumes that you've already run `setup lsst_distrib` to set up the LSST software stack environment.
    

First create `my_uri`, a string that defines which parts of the Data Release Pipeline (DRP) will be included.

To create a custom coadd, only two steps need to be rerun: `makeWarp` and `assembleCoadd`. 

In the file `$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml`, you will find these defined as the first two sub-steps of "step3", and that they are usually followed by the sub-step `detection`. 

However, we omit that sub-step and do source detection and measurement in the next tutorial notebook in this series, 09b_Custom_Coadd_Sources.ipynb.

In [ ]:
yaml_file = '$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml'
steps = 'makeWarp,assembleCoadd'
my_uri = yaml_file + '#' + steps
print(my_uri)

Next create a pipeline including just `makeWarp` and `assembleCoadd` steps, and call it `assembleCoaddPipeline`.

In [ ]:
assembleCoaddPipeline = Pipeline.from_uri(my_uri)

## 3.4. Configure the pipeline

There is only one configuration that must be set, and it is to clarify to the pipeline that it _does not_ need to redo a final image characterization step.
Currently, this configuration is only needed due to a version mismatch: the DP0.2 data sets were processed with Version 23 of the LSST Science Pipelines, whereas this notebook uses the version "Weekly 2022_40".

This configuration might not be needed in the future, and in general, users of this tutorial do not need to worry about other configuration changes. Just as an FYI, configurations can be set using `addConfigOverride(<taskName>, <configName>, <configValue>)`.
Section 4.2.5. shows how to view the configurations that were used for a given run.

In [ ]:
assembleCoaddPipeline.addConfigOverride('makeWarp', 'doApplyFinalizedPsf', False)

In [ ]:
assembleCoaddPipeline.addConfigOverride('makeWarp', 
                                        'connections.visitSummary', 
                                        "visitSummary")

## 3.5 Visualize the pipeline `QuantumGraph`

Before actually deploying the custom coaddition, let’s take a moment to inspect the `QuantumGraph` of the processing to be run. The QuantumGraph is <a href="https://pipelines.lsst.io/py-api/lsst.pipe.base.QuantumGraph.html#lsst.pipe.base.QuantumGraph">a tool</a> used by the LSST Science Pipelines to break a large processing into relatively “bite-sized” quanta and arrange these quanta into a sequence such that all inputs needed by a given quantum are available for the execution of that quantum. In the present case, you will not be doing an especially large processing, but for production deployments it makes sense to inspect and validate the `QuantumGraph` before proceeding straight to full-scale processing launch. The image below provides a visualization of the custom coadd processing's `QuantumGraph`.

<img align="left" src = https://dp0-2.lsst.io/_images/makeWarpAssembleCoadd.png width="100%" style="padding: 10px">

Light gray rectangles with rounded corners represent data, whereas darker gray rectangles with sharp corners represent pipeline tasks. The arrows connecting the data and tasks illustrate the data processing flow. The data processing starts at the top, with the `calexp` calibrated single-exposure images (also known as Processed Visit Images; PVIs). The `makeWarp` task is applied to generate reprojected "warp" images from the various input `calexp` images, and finally the `assembleCoadd` task combines the warps into `deepCoadd` coadded products (light gray boxes along the bottom row).

**Option:** recreate the above `QuantumGraph` visualization yourself. Note that running the following optional commands will create two files, `custom_coadd_qgraph.dot` and `custom_coadd_qgraph.png`, in your RSP home directory.

In [ ]:
# from lsst.ctrl.mpexec import pipeline2dot
# pipeline2dot(assembleCoaddPipeline, '/home/' + getpass.getuser() + '/custom_coadd_qgraph.dot')

# ! dot -Tpng /home/$USER/custom_coadd_qgraph.dot > /home/$USER/custom_coadd_qgraph.png

## 3.6. Create the full query string

Above, the visits to be recombined were stored in `my_visits_tupleString`.

Below, the full query string is built to include the patch being recreated.

In [ ]:
queryString = f"tract = {my_tract} AND patch = {my_patch} AND " + \
              f"visit in {my_visits_tupleString} AND skymap = 'DC2'"

print(queryString)

## 3.7. Use the Simple Pipeline Executor to run the pipeline

The <a href="https://pipelines.lsst.io/v/weekly/py-api/lsst.ctrl.mpexec.SimplePipelineExecutor.html">Simple Pipeline Executor</a> is a simple, high-level executor for pipelines that is primarily used for unit tests and small-scale integration of `PipelineTask` classes. It is appropriate to use in this case, where we are doing small-scale pipeline integration, in this case, of the `makeWarp` and `assembleCoaddPipeline` tasks only.

Learn more about the Simple Pipeline Executor.

In [ ]:
# SimplePipelineExecutor.from_pipeline?

Set up the Simple Pipeline Executor. This step can take up to 15 minutes.

In [ ]:
spe = SimplePipelineExecutor.from_pipeline(assembleCoaddPipeline, 
                                           where=queryString, 
                                           butler=simpleButler)

**Option:** Further explore the `QuantumGraph` via your `SimplePipelineExecutor` object. To see a pop-up window of `QuantumGraph`-related methods/attributes accessible via your `SimplePipelineExecutor`, put your cursor after the period and press the tab key.

In [ ]:
# spe.quantum_graph.

Run the pipeline.

There will be a lot of standard output. Alt-click to the left of the cell (or control-click for Macs) and choose "Enable Scrolling for Outputs" to condense all of the output into a scrollable inset window.

This takes about 20 minutes to coadd the 6 visits of Window1.

>**Notice:** Recall from Section 3.2 that the Butler will not overwrite existing data.
If you've already executed the cell below, and then you attempt to re-execute it without first re-executing the cell in Section 3.2 that instantiates the `simpleButler` with an output collection that has a unique timestamp, an error message will be returned.

In [ ]:
quanta = spe.run()

# 4. Display and analyze the results

## 4.1. The quanta

The `quanta` that was just created is a list of objects corresponding to each of the pipeline tasks that were run.
There will be one for each visit’s `makeWarp` and one for `assembleCoadd`.
Each of those has information about, e.g., the name of the task, the reference to the input and output.

**Option:** learn more about the `quanta`.

In [ ]:
# quanta

**Option:** to see the pop-up window of methods for `quanta`, put your cursor after the period and press the tab key.

In [ ]:
# quanta.

**Option:** instead of using tab, see the methods associated with an object by using `dir`.

In [ ]:
# dir(quanta[0])

Display the task names for each `quantum` in `quanta` to find the one associated with the `assembleCoadd` task.

In [ ]:
for q, quantum in enumerate(quanta):
    print(q, quantum.taskName)

When this tutorial was prepared, the output of the cell above showed that the first six elements of `quanta` (indices 0 through 5) were the results of `makeWarp` for the six visits, and then the seventh (index 6) was the results of `assembleCoadd`.

Below, set `coadd_index` equal to the index for the outputs of `lsst.pipe.tasks.assembleCoadd`.

This should be the same unless you've changed how many visits are input to the custom coadd.

> E.g., `coadd_index = 6`

In [ ]:
coadd_index = 6

**Option:** display information using the `inputs` and `outputs` methods for `quanta`.

In [ ]:
# quanta[coadd_index].inputs

In [ ]:
# quanta[coadd_index].outputs

In [ ]:
for output in quanta[coadd_index].outputs:
    print(output)

The `deepCoadd` is the output we're interested in. Notice that it has a `DatasetType` of `ExposureF`, so it is the image.

In [ ]:
quanta[coadd_index].outputs['deepCoadd']

From the square brackets, it's a list of just one element. So the `DatasetRef` for just our new coadd would be:

> `quanta[coadd_index].outputs['deepCoadd'][0]`

The above output is used in Section 4.2.1, below, to retrieve the new custom `deepCoadd` image from the `simpleButler`.

## 4.2. The new image

Although the new custom `deepCoadd` that we created is not actually deep, but a rather shallower custom coadd, it will still be called `deepCoadd` in the butler because that is the default name of results from the `assembleCoadd` task.

> **Notice:** Only use 4.2.1 "Option 1" ***OR*** 4.2.2 "Option 2" to retrieve the new image.

### 4.2.1. Option 1 - use the quanta

In [ ]:
my_new_deepCoadd = simpleButler.get(quanta[coadd_index].outputs['deepCoadd'][0])

### 4.2.2. Option 2 - use the dataId

The `dataId` for the `deepCoadd` of interest was already defined:
> `my_dataId = {'band': 'i', 'tract': my_tract, 'patch': my_patch}`

Use it to retrieve the newly made `deepCoadd` from the `simpleButler`.

So long as you use the `simpleButler`, it will search the collections in order, and since we made sure that the new collection we just created and wrote to appears first -- as checked in the first cell below -- it will take the `deepCoadd` from that new collection.

In [ ]:
# simpleButler.registry.getCollectionChain(my_outputCollection)

In [ ]:
# my_new_deepCoadd = simpleButler.get('deepCoadd', dataId=my_dataId)

#### Accessing the image from another notebook.

The results have been permanently written to the butler.

If you instantiated a new butler in a new notebook: <br>
`collection = "u/<Your User Name>/coadd_recreation_nb/Window1_coadd/<Unique Timestamp>"`<br>
`butler = Butler('dp02-direct', collections=collection)`<br>

Then you would be able to define the dataId in the same way:<br>
`my_dataId = {'band': 'i', 'tract': 4431, 'patch': 17}`<br>

And retrieve your new `deepCoadd` with:<br>
`my_new_deepCoadd = Butler.get('deepCoadd', dataId=my_dataId)`.

Since we already have our collection in the `simpleButler`, we don't need this option in this notebook.

This will be demonstrated in tutorial notebook 09b_Custom_Coadd_Sources.ipynb.

### 4.2.3. Check the inputs of the new Coadd

Check the inputs of `my_new_deepCoadd`, and compare them with the "queryString", and see that they match.

> **Notice:** With Weekly 41 it should be possible to do: <br>
> `my_new_deepCoadd_inputs = simpleButler.get("deepCoadd.coaddInputs", my_dataId, storageClass="AstropyTable")` <br>
> to have the results returned as an astropy table with a single command, instead of the two below.

In [ ]:
my_new_deepCoadd_inputs = simpleButler.get("deepCoadd.coaddInputs", my_dataId)

In [ ]:
my_new_deepCoadd_inputs.visits.asAstropy()

### 4.2.4. Display the new Coadd

In the next tutorial notebook in the series, 09b_Custom_Coadd_Sources.ipynb, the new custom `deepCoadd` is displayed alongside the original `deepCoadd` (along with other comparisons between the two).

Here, just show the newly coadded image.

In [ ]:
fig = plt.figure(figsize=(10, 6))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(my_new_deepCoadd.image)
plt.gca().axis('off')

### 4.2.5. Retrieve the configurations for the run

In this example, only one configuration parameter was changed from the default values. This was in Section 3.4 where we set:
`assembleCoaddPipeline.addConfigOverride('makeWarp', 'doApplyFinalizedPsf', False)`.

The configurations can be retrieved for the `simpleButler` with the following calls.

In [ ]:
my_makeWarp_config = simpleButler.get("makeWarp_config", my_dataId)

In [ ]:
my_assembleCoadd_config = simpleButler.get("assembleCoadd_config", my_dataId)

**Option:** print all of the keys and values for each pipeline tasks's configuration.

In [ ]:
# for key, value in my_makeWarp_config.items():
#     print(key, value)

In [ ]:
# for key, value in my_assembleCoadd_config.items():
#     print(key, value)

## 4.3. Sources in the new image

Use the next tutorial notebook in the series (09b_Custom_Coadd_Sources.ipynb) to do source detection and measurement on the new custom `deepCoadd`, and compare with results from the original `deepCoadd`.


# 5. Exercises for the learner

1. Create a custom `deepCoadd` for date Window2. Restart the kernel and clear all outputs. Edit Section 3 to use Window2. Note that when you get to re-executing Section 4.1 for Window2, the coadd_index will be different.
2. Use airmass constraints instead of MJD to identify the subset of visits to coadd. Hint: use `my_coadd_visits_airmass = visitTable.loc[my_coadd_inputs.visits['id']]['airmass']`.